In [1]:
from sklearn.linear_model import LogisticRegression
from preprocessor import Preprocessor
from evaluator import ModelEvaluator
import pandas as pd
import numpy as np

In [2]:
'''
Логарифмируем числовые фичи
'''

'\nЛогарифмируем числовые фичи\n'

In [3]:
df_train = pd.read_csv("C:/Users/Роман/Chepenkov_dz2/application_train.csv")
df_test = pd.read_csv("C:/Users/Роман/Chepenkov_dz2/application_test.csv")

In [4]:
def hypothesis_3(df):
   
    df = df.copy()
    
    num_cols = df.select_dtypes(include=['float64', 'int64']).columns
    
    for col in num_cols:
        if col in df.columns:
            min_val = df[col].min()
            shift = 1 - min_val if min_val <= 0 else 0
            df[col] = np.log1p(df[col] + shift)
    
    return df

In [5]:
def hypothesis_3_1(df):
   
    df = df.copy()
    
    num_cols = df.select_dtypes(include=['float64', 'int64']).columns
    
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df[num_cols])
    
    return df_scaled

In [6]:
preprocessor = Preprocessor(custom_steps=[hypothesis_3])

X_train_processed = preprocessor.transform(df_train.drop(columns=['TARGET', 'SK_ID_CURR']))
num_cols = X_train_processed.select_dtypes(include='number').columns
X_train_processed = X_train_processed[num_cols]

X_test_processed = preprocessor.transform(df_test.drop(columns=['SK_ID_CURR']))
X_test_processed = X_test_processed[num_cols]

MemoryError: Unable to allocate 152. MiB for an array with shape (65, 307511) and data type float64

In [ ]:
evaluator = ModelEvaluator(LogisticRegression(max_iter=100))

roc_auc = evaluator.evaluate(X_train_processed, df_train['TARGET'])
print(f"ROC-AUC train: {roc_auc:.5f}")

evaluator.save_test_predictions(
    X_test=X_test_processed,
    ids=df_test['SK_ID_CURR'],
    filename='test_predictions_hyp_3.csv'
)
#ROC-AUC train log: 0.73342

#ROC-AUC test: 0.71731
#ROC-AUC test baseline: 0.61204
#ROC-AUC test surplus: 0.10527